# MEA

In [1]:
from CodeLibrary import Simulation
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


## Defining the name of streams and blocks in the simulation
strm_leansolv = "LEANSOLV" 
strm_cleangas = "CLEANGAS"
strm_fluegas = "FLUEGAS"
strm_co2 = "CO2"
amine_comp = "MEA"     # Type of amine used in the simulation
blk_stripper = "STRIPPER"
blk_cooler = "COOLER"
blk_makeup = "MAKEUP"
calc_co2rec = "CO2REC" # Calculator block consist of CO2 Recovery%
calc_sec = "ENERGY"    # Calculator block consist of SEC (kJ/kg)

## Defining empty lists
moleflowCO2_fluegas_List = []   # {kmol/h}
moleflowMEA_leansolv_List = []  # {kmol/h} 
# The following features are considered
molefracCO2_fluegas_List = []
massflow_fluegas_List = []      # {kg/h}
massflow_leansolv_List = []     # {kg/h}
massfracAmine_leansolv_List = []
loading = []
temp_leansolv_List = []
duty_reb_List = []
# Targets
recovery = []  # CO2 Recovery %
sec = []       # Specific Energy Comsumption (kJ/kg co2 captured)
purity = []    # CO2 Mole Fraction in the Stream Exiting the Stripper's Condenser

## Aspen and directory paths
AspenPath = r"D:/daneshgah/undergrad/Term9/Carbon Capture/Python/MEA" # Path of the Aspen file in your system 
WorkDirect = r"C:/Users/Pgshco/Desktop/Python ML/Carbon Capture"  # The directory you are coding in
sim = Simulation(AspenFileName= AspenPath + "/MEA.bkp", WorkingDirectoryPath= WorkDirect,VISIBILITY=True) 
# Choose the "Visibility" as "False" if you don't want the Aspen app be visible during the process

# Output dataset path
out_data = AspenPath + "/MEAdatanew.xlsx"
# Desired column order in the Excel or CSV
COLUMNS = [
    # features
    'Fluegas CO2 mol%',
    'Fluegas Flow (kg/h)',
    'Lean Solvent Flow (kg/h)', 
    'Lean Solvent Loading',
    'Amine type',
    'Amine Conc (wt%)',
    'Lean Solvent T (C)',
    'Reboiler Duty (MW)',
    # Targets
    'Recovery%',
    'SEC (kJ/kg)',
    'CO2 Purity (mol%)',
]
rows = []

## list of features ranges in the simulation
FlueFlow = [1.5e6, 3.1e6, 3.25e6, 3.4e6, 3.6e6]
# Fluegas Composition {columns order: H2O, CO2, N2, O2}
FlueComp = [[0.1156, 0.020, 0.7016, 0.1628],    # 0.02, 0.025, 0.03, 0.035, 0.04, 0.045
            [0.115, 0.025, 0.698, 0.162],
            [0.1144, 0.03, 0.6944, 0.1612],
            [0.1138, 0.035, 0.6909, 0.1603],
            [0.1132, 0.04, 0.6873, 0.1595],
            [0.1126, 0.045, 0.6837, 0.1587]]  
AmineConc = [0.15, 0.17, 0.19, 0.22, 0.25]                      
LeanFlow = [0.8e+6, 1e6, 1.2e+6, 1.4e6, 1.6e6, 1.8e6, 2.2e6]   

# FlueFlow = [1.66e6, 2.4e6, 3.39e6]
# # Fluegas Composition {columns order: H2O, CO2, N2, O2}
# FlueComp = [[0.1156, 0.020, 0.7016, 0.1628],    # 0.02, 0.025, 0.03, 0.035, 0.04, 0.045
#             # [0.1144, 0.03, 0.6944, 0.1612],
#             [0.1126, 0.045, 0.6837, 0.1587]]  
# AmineConc = [0.18, 0.24]                      
# LeanFlow = [1.35e6, 1.92e6] 
LeanTemp = [30, 50, 60]
RebDuty = [245, 400, 480, 510]

iter = 0

for duty_reb in RebDuty:
    for flow_lean in LeanFlow:             # Manipulated from "MAKEUP" block
        for conc_amine in AmineConc:       # Manipulated from "MAKEUP" block
            for flow_flue in FlueFlow:
                for comp_flue in FlueComp: # Normalized composition 
                    for temp_lean in LeanTemp:
                        ## Setting the values in the simulation
                        sim.BLK_MAKEUP_Set_MassFlow(blk_makeup, flow_lean)
                        sim.BLK_MAKEUP_Set_MassFrac(blk_makeup, conc_amine, 0)
                        sim.STRM_Set_TotalFlowRate(strm_fluegas, flow_flue)
                        # sim.BLK_HEATER_Set_FlashTypeOption(blk_cooler,"TP")
                        sim.BLK_COOLER_TP_Set_Temperature(blk_cooler, temp_lean)
                        sim.BLK_RADFRAC_Set_ReboilerDuty(blk_stripper, duty_reb)
                        # FLUEGAS composition
                        sim.STRM_Set_ComponentFlowRate(strm_fluegas,comp_flue[0],"H2O")
                        sim.STRM_Set_ComponentFlowRate(strm_fluegas,comp_flue[1],"CO2")
                        sim.STRM_Set_ComponentFlowRate(strm_fluegas,comp_flue[2],"N2")   
                        sim.STRM_Set_ComponentFlowRate(strm_fluegas,comp_flue[3],"O2")
                    
                        ## Print the current iteration inputs
                        iter += 1
                        print(f"\n \niteration:{iter} "
                              f"\n FLUEGAS massflow: {flow_flue} kg/h "
                              f"\n FLUEGAS CO2 mole frac: {comp_flue[1]} "
                              f"\n LEANSOL massflow: {flow_lean} kg/h "
                              f"\n Amine concentration (wt%): {conc_amine}"
                              f"\n LEANSOL temperature: {temp_lean} C "
                              f"\n Reboiler Duty: {duty_reb} MW ")
                        
                        ## Running the simulation
                        #Suppress dialog (This is set to be True due to initialization)
                        sim.DialogSuppression(TrueOrFalse= True)
                        time_out = 60  # set the timeout to stop the simulation and move to the next iteration -> unit{seconds}
                        convergence = sim.RunWithTimeout(time_out)     
                        if convergence == True:
                            print("The simulation ran successfully.")
                        else:
                            print(f"The simulation didn't converged in {time_out}s!")
                            continue
                        
                        ## Getting the results from the simulation
                        # CO2 moleflow input (in the FLUEGAS stream)
                        moleflowCO2_fluegas_List.append(sim.STRM_Get_MoleFlowPerCompound(strm_fluegas,"CO2"))
                        # Input MEA moleflow (in the LEANSOLV stream)
                        moleflowMEA_leansolv_List.append(sim.STRM_Get_MoleFlowPerCompound(strm_leansolv,"MEA"))
                        # CO2-Amine Loading
                        loading.append(moleflowCO2_fluegas_List[-1]/ moleflowMEA_leansolv_List[-1])
                        # Fluegas Massflow
                        massflow_fluegas_List.append(flow_flue)
                        # CO2 mole fraction
                        molefracCO2_fluegas_List.append(comp_flue[1])
                        # Amine concentration in the LEANSOLV
                        massfracAmine_leansolv_List.append(sim.STRM_Get_MassFracPerCompound(strm_leansolv,"MEA"))
                        # Lean solvent mass flow (kg/h)
                        massflow_leansolv_List.append(sim.STRM_Get_MassFlowPerCompound(strm_leansolv,"MEA") / massfracAmine_leansolv_List[-1])
                        # Lean solvent Temperature (C)
                        temp_leansolv_List.append(temp_lean)
                        # Reboiler Duty
                        duty_reb_List.append(duty_reb)
                        # Targets
                        recovery.append(sim.CALC_Get_OutputVal("CO2REC")) 
                        sec.append(sim.CALC_Get_OutputVal("ENERGY"))
                        purity.append(sim.STRM_Get_MoleFracPerCompound(strm_co2,"CO2") * 100)
                        # Print the targets
                        print("The CO2 Recovery%: ", recovery[-1])
                        print("The Specific Energy Consumption (kJ/kg)): ", sec[-1])
                        print("The CO2 purity%: ", purity[-1]) 
                        
                        ## Adding the data to the dictionary 
                        row = {
                            'Fluegas CO2 mol%':           100 * molefracCO2_fluegas_List[-1],
                            'Fluegas Flow (kg/h)':        massflow_fluegas_List[-1],
                            'Lean Solvent Flow (kg/h)':   massflow_leansolv_List[-1],
                            'Lean Solvent Loading':       loading[-1], # constant for now
                            'Amine type':                 amine_comp,
                            'Amine Conc (wt%)':           100 * massfracAmine_leansolv_List[-1],
                            'Lean Solvent T (C)':         temp_leansolv_List[-1],
                            'Reboiler Duty (MW)':         duty_reb_List[-1],
                            'Recovery%':                  recovery[-1],
                            'SEC (kJ/kg)':                sec[-1],
                            'CO2 Purity (mol%)':          purity[-1],
                        }
                        rows.append(row)       
                    
            
sim.CloseAspen()

## Saving to an Excel file
df = pd.DataFrame(rows)[COLUMNS]
df.to_excel(out_data, index=False)
print(f"\nSaved dataset to: {out_data}")

The current Directory is :  
C:\Users\Pgshco\Desktop\Python ML\Carbon Capture
The new Directory where you should also have your Aspen file is : 
C:\Users\Pgshco\Desktop\Python ML\Carbon Capture
The Aspen is active now. If you dont want to see aspen open again take VISIBITLY as False 


 
iteration:73 
 FLUEGAS massflow: 1660000.0 kg/h 
 FLUEGAS CO2 mole frac: 0.02 
 LEANSOL massflow: 1350000.0 kg/h 
 Amine concentration (wt%): 0.18
 LEANSOL temperature: 30 C 
 Reboiler Duty: 400 MW 
Runtime = 50.18249845504761
The simulation ran successfully.
The CO2 Recovery%:  99.9132664
The Specific Energy Consumption (kJ/kg)):  51321.0524
The CO2 purity%:  24.0913663

 
iteration:74 
 FLUEGAS massflow: 1660000.0 kg/h 
 FLUEGAS CO2 mole frac: 0.02 
 LEANSOL massflow: 1350000.0 kg/h 
 Amine concentration (wt%): 0.18
 LEANSOL temperature: 50 C 
 Reboiler Duty: 400 MW 
Runtime = 10.193682432174683
The simulation ran successfully.
The CO2 Recovery%:  99.8936655
The Specific Energy Consumption (kJ/kg)): 

In [29]:
# random values for features
import random 

random.seed(46)
a, b = int(1.5e6), int(3e6)
mid = (a + b) // 2
nums = sorted([random.randint(a, mid), random.randint(mid, b)])
print(nums)

[1580093, 2669424]


# DGA

In [1]:
from CodeLibrary import Simulation
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Defining the name of streams and blocks in the simulation
strm_leansolv = "LEANSOLV" 
strm_cleangas = "CLEANGAS"
strm_fluegas = "FLUEGAS"
strm_co2 = "CO2"
amine_comp = "DGA"     # Type of amine used in the simulation
blk_stripper = "STRIPPER"
blk_cooler = "COOLER"
blk_makeup = "MAKEUP"
calc_co2rec = "CO2REC" # Calculator block consist of CO2 Recovery%
calc_sec = "ENERGY"    # Calculator block consist of SEC (kJ/kg)

## Defining empty lists
moleflowCO2_fluegas_List = []   # {kmol/h}
moleflowDGA_leansolv_List = []  # {kmol/h} 
# The following features are considered
molefracCO2_fluegas_List = []
massflow_fluegas_List = []      # {kg/h}
massflow_leansolv_List = []     # {kg/h}
massfracAmine_leansolv_List = []
loading = []
temp_leansolv_List = []
duty_reb_List = []
# Targets
recovery = []  # CO2 Recovery %
sec = []       # Specific Energy Comsumption (kJ/kg co2 captured)
purity = []    # CO2 Mole Fraction in the Stream Exiting the Stripper's Condenser

## Aspen and directory paths
AspenPath = r"D:/daneshgah/undergrad/Term9/Carbon Capture/Python/DGA" # Path of the Aspen file in your system 
WorkDirect = r"C:/Users/Pgshco/Desktop/Python ML/Carbon Capture"  # The directory you are coding in
sim = Simulation(AspenFileName= AspenPath + "/DGA.bkp", WorkingDirectoryPath= WorkDirect,VISIBILITY=True) 
# Choose the "Visibility" as "False" if you don't want the Aspen app be visible during the process

# Output dataset path
out_data = AspenPath + "/DGAdatanew.xlsx"
# Desired column order in the Excel or CSV
COLUMNS = [
    'Fluegas CO2 mol%',
    'Fluegas Flow (kg/h)',
    'Lean Solvent Flow (kg/h)', 
    'Lean Solvent Loading',
    'Amine type',
    'Amine Conc (wt%)',
    'Lean Solvent T (C)',
    'Reboiler Duty (MW)',
    # Targets
    'Recovery%',
    'SEC (kJ/kg)',
    'CO2 Purity (mol%)',
]
rows = []

# ## list of features ranges in the simulation
# FlueFlow = [1.5e6, 2.1e6, 2.7e6, 3.1e6, 3.4e6]
# # Fluegas Composition {columns order: H2O, CO2, N2, O2}
# FlueComp = [[0.1156, 0.020, 0.7016, 0.1628],    # 0.02, 0.025, 0.03, 0.035, 0.04, 0.045
#             [0.1149, 0.026, 0.6973, 0.1618],
#             [0.1142, 0.032, 0.693, 0.1608],
#             [0.1135, 0.038, 0.6887, 0.1598],
#             [0.1126, 0.045, 0.6837, 0.1587]]  
# AmineConc = [0.25, 0.3, 0.37, 0.46, 0.55]                      
# LeanFlow = [1e6, 1.3e+6, 1.6e6, 1.9e6, 2.5e6]         

FlueFlow = [1.7e6, 2.8e6, 3.2e6]
# Fluegas Composition {columns order: H2O, CO2, N2, O2}
FlueComp = [[0.1149, 0.026, 0.6973, 0.1618],
            [0.1135, 0.038, 0.6887, 0.1598]]  
AmineConc = [0.27, 0.47]                      
LeanFlow = [1.2e+6, 2.1e6]         
LeanTemp = [25, 40, 60]
RebDuty = [245, 400, 480, 510]

iter = 0

for duty_reb in RebDuty:
    for flow_lean in LeanFlow:             # Manipulated from "MAKEUP" block
        for conc_amine in AmineConc:       # Manipulated from "MAKEUP" block
            for flow_flue in FlueFlow:
                for comp_flue in FlueComp: # Normalized composition 
                    for temp_lean in LeanTemp:
                        ## Setting the values in the simulation
                        sim.BLK_MAKEUP_Set_MassFlow(blk_makeup, flow_lean)
                        sim.BLK_MAKEUP_Set_MassFrac(blk_makeup, conc_amine, 0)
                        sim.STRM_Set_TotalFlowRate(strm_fluegas, flow_flue)
                        # sim.BLK_HEATER_Set_FlashTypeOption(blk_cooler,"TP")
                        sim.BLK_COOLER_TP_Set_Temperature(blk_cooler, temp_lean)
                        sim.BLK_RADFRAC_Set_ReboilerDuty(blk_stripper, duty_reb)
                        # FLUEGAS composition
                        sim.STRM_Set_ComponentFlowRate(strm_fluegas,comp_flue[0],"H2O")
                        sim.STRM_Set_ComponentFlowRate(strm_fluegas,comp_flue[1],"CO2")
                        sim.STRM_Set_ComponentFlowRate(strm_fluegas,comp_flue[2],"N2")   
                        sim.STRM_Set_ComponentFlowRate(strm_fluegas,comp_flue[3],"O2")
                    
                        ## Print the current iteration inputs
                        iter += 1
                        print(f"\n \niteration:{iter} "
                              f"\n FLUEGAS massflow: {flow_flue} kg/h "
                              f"\n FLUEGAS CO2 mole frac: {comp_flue[1]} "
                              f"\n LEANSOL massflow: {flow_lean} kg/h "
                              f"\n Amine concentration (wt%): {conc_amine}")
                        
                        ## Running the simulation
                        #Suppress dialog (This is set to be True due to initialization)
                        sim.DialogSuppression(TrueOrFalse= True)
                        time_out = 70  # set the timeout to stop the simulation and move to the next iteration -> unit{seconds}
                        convergence = sim.RunWithTimeout(time_out)     
                        if convergence == True:
                            print("The simulation ran successfully.")
                        else:
                            print(f"The simulation didn't converged in {time_out}s!")
                            continue
                        
                        ## Getting the results from the simulation
                        # CO2 moleflow input (in the FLUEGAS stream)
                        moleflowCO2_fluegas_List.append(sim.STRM_Get_MoleFlowPerCompound(strm_fluegas,"CO2"))
                        # Input Amine moleflow (in the LEANSOLV stream)
                        moleflowDGA_leansolv_List.append(sim.STRM_Get_MoleFlowPerCompound(strm_leansolv,amine_comp))
                        # CO2-Amine Loading
                        loading.append(moleflowCO2_fluegas_List[-1]/ moleflowDGA_leansolv_List[-1])
                        # Fluegas Massflow
                        massflow_fluegas_List.append(flow_flue)
                        # CO2 mole fraction 
                        molefracCO2_fluegas_List.append(comp_flue[1])
                        # Amine concentration in the LEANSOLV
                        massfracAmine_leansolv_List.append(sim.STRM_Get_MassFracPerCompound(strm_leansolv,amine_comp))
                        # Lean solvent mass flow (kg/h)
                        massflow_leansolv_List.append(sim.STRM_Get_MassFlowPerCompound(strm_leansolv,amine_comp) / massfracAmine_leansolv_List[-1])
                        # Lean solvent Temperature (C)
                        temp_leansolv_List.append(temp_lean)
                        # Reboiler Duty
                        duty_reb_List.append(duty_reb)
                        # Targets
                        recovery.append(sim.CALC_Get_OutputVal("CO2REC")) 
                        sec.append(sim.CALC_Get_OutputVal("ENERGY"))
                        purity.append(sim.STRM_Get_MoleFracPerCompound(strm_co2,"CO2") * 100)
                        # Print the targets
                        print("The CO2 Recovery%: ", recovery[-1])
                        print("The Specific Energy Consumption (kJ/kg)): ", sec[-1])
                        print("The CO2 purity%: ", purity[-1]) 
                        
                        ## Adding the data to the dictionary 
                        row = {
                            'Fluegas CO2 mol%':           100 * molefracCO2_fluegas_List[-1],
                            'Fluegas Flow (kg/h)':        massflow_fluegas_List[-1],
                            'Lean Solvent Flow (kg/h)':   massflow_leansolv_List[-1],
                            'Lean Solvent Loading':       loading[-1], # constant for now
                            'Amine type':                 amine_comp,
                            'Amine Conc (wt%)':           100 * massfracAmine_leansolv_List[-1],
                            'Lean Solvent T (C)':         temp_leansolv_List[-1],
                            'Reboiler Duty (MW)':         duty_reb_List[-1],
                            'Recovery%':                  recovery[-1],
                            'SEC (kJ/kg)':                sec[-1],
                            'CO2 Purity (mol%)':          purity[-1],
                        }
                        rows.append(row)       
                        # time.sleep(0.1) # Better convergence and data saving
            
sim.CloseAspen()

## Saving to an Excel file
df = pd.DataFrame(rows)[COLUMNS]
df.to_excel(out_data, index=False)
print(f"\nSaved dataset to: {out_data}")

The current Directory is :  
C:\Users\Pgshco\Desktop\Python ML\Carbon Capture
The new Directory where you should also have your Aspen file is : 
C:\Users\Pgshco\Desktop\Python ML\Carbon Capture
The Aspen is active now. If you dont want to see aspen open again take VISIBITLY as False 


 
iteration:1 
 FLUEGAS massflow: 1700000.0 kg/h 
 FLUEGAS CO2 mole frac: 0.026 
 LEANSOL massflow: 1200000.0 kg/h 
 Amine concentration (wt%): 0.27
Runtime so far = 70.1 s
⏱ Timeout > 70s, stopping run...
Runtime so far = 77.5 s
⏱ Timeout > 70s, stopping run...
Runtime = 89.4364881515503
The simulation didn't converged in 70s!

 
iteration:2 
 FLUEGAS massflow: 1700000.0 kg/h 
 FLUEGAS CO2 mole frac: 0.026 
 LEANSOL massflow: 1200000.0 kg/h 
 Amine concentration (wt%): 0.27
Runtime = 45.27927756309509
The simulation ran successfully.
The CO2 Recovery%:  77.4422752
The Specific Energy Consumption (kJ/kg)):  27872.6959
The CO2 purity%:  25.017676599999998

 
iteration:3 
 FLUEGAS massflow: 1700000.0 kg/h 

# DEA

In [1]:
from CodeLibrary import Simulation
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Defining the name of streams and blocks in the simulation
strm_leansolv = "LEANSOLV" 
strm_cleangas = "CLEANGAS"
strm_fluegas = "FLUEGAS"
strm_co2 = "CO2"
amine_comp = "DEA"     # Type of amine used in the simulation
blk_stripper = "STRIPPER"
blk_cooler = "COOLER"
blk_makeup = "MAKEUP"
calc_co2rec = "CO2REC" # Calculator block consist of CO2 Recovery%
calc_sec = "ENERGY"    # Calculator block consist of SEC (kJ/kg)

## Defining empty lists
moleflowCO2_fluegas_List = []   # {kmol/h}
moleflowDEA_leansolv_List = []  # {kmol/h} 
notConverged_itr = []
# The following features are considered
molefracCO2_fluegas_List = []
massflow_fluegas_List = []      # {kg/h}
massflow_leansolv_List = []     # {kg/h}
massfracAmine_leansolv_List = []
loading = []
temp_leansolv_List = []
duty_reb_List = []
# Targets
recovery = []  # CO2 Recovery %
sec = []       # Specific Energy Comsumption (kJ/kg co2 captured)
purity = []    # CO2 Mole Fraction in the Stream Exiting the Stripper's Condenser

## Aspen and directory paths
AspenPath = r"D:/daneshgah/undergrad/Term9/Carbon Capture/Python/DEA" # Path of the Aspen file in your system 
WorkDirect = r"C:/Users/Pgshco/Desktop/Python ML/Carbon Capture"  # The directory you are coding in
sim = Simulation(AspenFileName= AspenPath + "/DEA.bkp", WorkingDirectoryPath= WorkDirect,VISIBILITY=True) 
# Choose the "Visibility" as "False" if you don't want the Aspen app be visible during the process

# Output dataset path
out_data = AspenPath + "/DEAdatanew.xlsx"
# Desired column order in the Excel or CSV
COLUMNS = [
    'Fluegas CO2 mol%',
    'Fluegas Flow (kg/h)',
    'Lean Solvent Flow (kg/h)', 
    'Lean Solvent Loading',
    'Amine type',
    'Amine Conc (wt%)',
    'Lean Solvent T (C)',
    'Reboiler Duty (MW)',
    # Targets
    'Recovery%',
    'SEC (kJ/kg)',
    'CO2 Purity (mol%)',
]
rows = []

## list of features ranges in the simulation
# FlueFlow = [1.5e6, 1.88e6, 2.26e6, 2.64e6, 3.02e6, 3.3e6] 
# # Fluegas Composition {columns order: H2O, CO2, N2, O2}
# FlueComp = [[0.1156, 0.020, 0.7016, 0.1628],    # 0.02, 0.025, 0.03, 0.035, 0.04, 0.045
#             [0.115, 0.025, 0.698, 0.162],
#             [0.1144, 0.03, 0.6944, 0.1612],
#             [0.1138, 0.035, 0.6909, 0.1603],
#             [0.1132, 0.04, 0.6873, 0.1595],
#             [0.1126, 0.045, 0.6837, 0.1587]]  
# AmineConc = [0.2, 0.275, 0.35, 0.425, 0.5]                
# LeanFlow = [8e+5, 1.2e+6, 1.6e+6, 2e+6, 2.4e6, 2.8e+6]    



# FlueFlow = [1.5e6, 1.88e6, 2.26e6, 2.64e6, 3.02e6, 3.3e6] 
# # Fluegas Composition {columns order: H2O, CO2, N2, O2}
# FlueComp = [[0.1156, 0.020, 0.7016, 0.1628],    # 0.02, 0.025, 0.03, 0.035, 0.04, 0.045
#             [0.115, 0.025, 0.698, 0.162],
#             [0.1144, 0.03, 0.6944, 0.1612],
#             [0.1138, 0.035, 0.6909, 0.1603],
#             [0.1132, 0.04, 0.6873, 0.1595],
#             [0.1126, 0.045, 0.6837, 0.1587]]  
# AmineConc = [0.2, 0.275, 0.35, 0.425, 0.5]                
# LeanFlow = [8e+5, 1.2e+6, 1.6e+6, 2e+6, 2.4e6, 2.8e+6] 
FlueFlow = [1.8e6, 2.8e6, 3.2e6] 
# Fluegas Composition {columns order: H2O, CO2, N2, O2}
FlueComp = [#[0.1156, 0.020, 0.7016, 0.1628],    # 0.02, 0.025, 0.03, 0.035, 0.04, 0.045
            # [0.115, 0.025, 0.698, 0.162],
            [0.1144, 0.03, 0.6944, 0.1612],
            # [0.1138, 0.035, 0.6909, 0.1603],
            [0.1132, 0.04, 0.6873, 0.1595]]
            # [0.1126, 0.045, 0.6837, 0.1587]]  
AmineConc = [0.33, 0.4]                
LeanFlow = [1.8e6, 2.5e+6] 
LeanTemp = [33, 48, 60]
RebDuty = [260, 390, 450, 500]


iter = 0

for duty_reb in RebDuty:
    for flow_lean in LeanFlow:             # Manipulated from "MAKEUP" block
        for conc_amine in AmineConc:       # Manipulated from "MAKEUP" block
            for flow_flue in FlueFlow:
                for comp_flue in FlueComp: # Normalized composition 
                    for temp_lean in LeanTemp:
                        iter += 1
                        if iter <= 156:
                            continue
                        ## Setting the values in the simulation
                        sim.BLK_MAKEUP_Set_MassFlow(blk_makeup, flow_lean)
                        sim.BLK_MAKEUP_Set_MassFrac(blk_makeup, conc_amine, 0)
                        sim.STRM_Set_TotalFlowRate(strm_fluegas, flow_flue)
                        # sim.BLK_HEATER_Set_FlashTypeOption(blk_cooler,"TP")
                        sim.BLK_COOLER_TP_Set_Temperature(blk_cooler, temp_lean)
                        sim.BLK_RADFRAC_Set_ReboilerDuty(blk_stripper, duty_reb)
                        # FLUEGAS composition
                        sim.STRM_Set_ComponentFlowRate(strm_fluegas,comp_flue[0],"H2O")
                        sim.STRM_Set_ComponentFlowRate(strm_fluegas,comp_flue[1],"CO2")
                        sim.STRM_Set_ComponentFlowRate(strm_fluegas,comp_flue[3],"N2")   
                        sim.STRM_Set_ComponentFlowRate(strm_fluegas,comp_flue[2],"O2")
                    
                        ## Print the current iteration inputs
                        # iter += 1
                        print(f"\n \niteration:{iter} "
                              f"\n FLUEGAS massflow: {flow_flue} kg/h "
                              f"\n FLUEGAS CO2 mole frac: {comp_flue[1]} "
                              f"\n LEANSOL massflow: {flow_lean} kg/h "
                              f"\n Amine concentration (wt%): {conc_amine}"
                              f"\n LEANSOL temperature: {temp_lean} C "
                              f"\n Reboiler Duty: {duty_reb} MW ")
                        
                        ## Running the simulation
                        #Suppress dialog (This is set to be True due to initialization)
                        sim.DialogSuppression(TrueOrFalse= True)
                        time_out = 120  # set the timeout to stop the simulation and move to the next iteration -> unit{seconds}
                        convergence = sim.RunWithTimeout(time_out)     
                        if convergence == True:
                            print("The simulation ran successfully.")
                        else:
                            print(f"The simulation didn't converged in {time_out}s!")
                            notConverged_itr.append(iter) # To know which iterations didn't converge
                            continue
                        
                        ## Getting the results from the simulation
                        # CO2 moleflow input (in the FLUEGAS stream)
                        moleflowCO2_fluegas_List.append(sim.STRM_Get_MoleFlowPerCompound(strm_fluegas,"CO2"))
                        # Input Amine moleflow (in the LEANSOLV stream)
                        moleflowDEA_leansolv_List.append(sim.STRM_Get_MoleFlowPerCompound(strm_leansolv,"DEA"))
                        # CO2-Amine Loading
                        loading.append(moleflowCO2_fluegas_List[-1]/ moleflowDEA_leansolv_List[-1])
                        # Fluegas Massflow
                        massflow_fluegas_List.append(flow_flue)
                        # CO2 mole fraction 
                        molefracCO2_fluegas_List.append(comp_flue[1])
                        # Amine concentration in the LEANSOLV
                        massfracAmine_leansolv_List.append(sim.STRM_Get_MassFracPerCompound(strm_leansolv,amine_comp))
                        # Lean solvent mass flow (kg/h)
                        massflow_leansolv_List.append(sim.STRM_Get_MassFlowPerCompound(strm_leansolv,amine_comp) / massfracAmine_leansolv_List[-1])
                        # Lean solvent Temperature (C)
                        temp_leansolv_List.append(temp_lean)
                        # Reboiler Duty
                        duty_reb_List.append(duty_reb)
                        # Targets
                        recovery.append(sim.CALC_Get_OutputVal("CO2REC")) 
                        sec.append(sim.CALC_Get_OutputVal("ENERGY"))
                        purity.append(sim.STRM_Get_MoleFracPerCompound(strm_co2,"CO2") * 100)
                        # Print the targets
                        print("The CO2 Recovery%: ", recovery[-1])
                        print("The Specific Energy Consumption (kJ/kg)): ", sec[-1])
                        print("The CO2 purity%: ", purity[-1]) 
                        
                        ## Adding the data to the dictionary 
                        row = {
                            'Fluegas CO2 mol%':           100 * molefracCO2_fluegas_List[-1],
                            'Fluegas Flow (kg/h)':        massflow_fluegas_List[-1],
                            'Lean Solvent Flow (kg/h)':   massflow_leansolv_List[-1],
                            'Lean Solvent Loading':       loading[-1], # constant for now
                            'Amine type':                 amine_comp,
                            'Amine Conc (wt%)':           100 * massfracAmine_leansolv_List[-1],
                            'Lean Solvent T (C)':         temp_leansolv_List[-1],
                            'Reboiler Duty (MW)':         duty_reb_List[-1],
                            'Recovery%':                  recovery[-1],
                            'SEC (kJ/kg)':                sec[-1],
                            'CO2 Purity (mol%)':          purity[-1],
                        }
                        rows.append(row)       
                    
            
sim.CloseAspen()

## Saving to an Excel file
df = pd.DataFrame(rows)[COLUMNS]
df.to_excel(out_data, index=False)
print(f"\nSaved dataset to: {out_data}")

The current Directory is :  
C:\Users\Pgshco\Desktop\Python ML\Carbon Capture
The new Directory where you should also have your Aspen file is : 
C:\Users\Pgshco\Desktop\Python ML\Carbon Capture
The Aspen is active now. If you dont want to see aspen open again take VISIBITLY as False 


 
iteration:157 
 FLUEGAS massflow: 3200000.0 kg/h 
 FLUEGAS CO2 mole frac: 0.03 
 LEANSOL massflow: 1800000.0 kg/h 
 Amine concentration (wt%): 0.33
 LEANSOL temperature: 33 C 
 Reboiler Duty: 450 MW 
Runtime = 40.06322503089905
The simulation ran successfully.
The CO2 Recovery%:  53.3290802
The Specific Energy Consumption (kJ/kg)):  26314.3173
The CO2 purity%:  40.340989799999996

 
iteration:158 
 FLUEGAS massflow: 3200000.0 kg/h 
 FLUEGAS CO2 mole frac: 0.03 
 LEANSOL massflow: 1800000.0 kg/h 
 Amine concentration (wt%): 0.33
 LEANSOL temperature: 48 C 
 Reboiler Duty: 450 MW 
Runtime = 15.134023427963257
The simulation ran successfully.
The CO2 Recovery%:  53.4937745
The Specific Energy Consumption 

In [5]:
## Saving to an Excel file
df = pd.DataFrame(rows)[COLUMNS]
df.to_excel(out_data, index=False)
print(f"\nSaved dataset to: {out_data}")


Saved dataset to: D:/daneshgah/undergrad/Term9/Carbon Capture/Python/DEA/DEAdatanew.xlsx


# TEA

In [1]:
from CodeLibrary import Simulation
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Defining the name of streams and blocks in the simulation
strm_leansolv = "LEANSOLV" 
strm_cleangas = "CLEANGAS"
strm_fluegas = "FLUEGAS"
strm_co2 = "CO2"
amine_comp = "TEA"     # Type of amine used in the simulation
blk_stripper = "STRIPPER"
blk_cooler = "COOLER"
blk_makeup = "MAKEUP"
calc_co2rec = "CO2REC" # Calculator block consist of CO2 Recovery%
calc_sec = "ENERGY"    # Calculator block consist of SEC (kJ/kg)

## Defining empty lists
moleflowCO2_fluegas_List = []   # {kmol/h}
moleflowTEA_leansolv_List = []  # {kmol/h} 
notConverged_itr = []
# The following features are considered
molefracCO2_fluegas_List = []
massflow_fluegas_List = []      # {kg/h}
massflow_leansolv_List = []     # {kg/h}
massfracAmine_leansolv_List = []
loading = []
temp_leansolv_List = []
duty_reb_List = []
# Targets
recovery = []  # CO2 Recovery %
sec = []       # Specific Energy Comsumption (kJ/kg co2 captured)
purity = []    # CO2 Mole Fraction in the Stream Exiting the Stripper's Condenser

## Aspen and directory paths
AspenPath = r"D:/daneshgah/undergrad/Term9/Carbon Capture/Python/TEA" # Path of the Aspen file in your system 
WorkDirect = r"C:/Users/Pgshco/Desktop/Python ML/Carbon Capture"  # The directory you are coding in
sim = Simulation(AspenFileName= AspenPath + "/TEA.bkp", WorkingDirectoryPath= WorkDirect,VISIBILITY=True) 
# Choose the "Visibility" as "False" if you don't want the Aspen app be visible during the process

# Output dataset path
out_data = AspenPath + "/TEAdatanew.xlsx"
# Desired column order in the Excel or CSV
COLUMNS = [
    'Fluegas CO2 mol%',
    'Fluegas Flow (kg/h)',
    'Lean Solvent Flow (kg/h)', 
    'Lean Solvent Loading',
    'Amine type',
    'Amine Conc (wt%)',
    'Lean Solvent T (C)',
    'Reboiler Duty (MW)',
    # Targets
    'Recovery%',
    'SEC (kJ/kg)',
    'CO2 Purity (mol%)',
]
rows = []

## list of features ranges in the simulation
# FlueFlow = [1e6, 1.6e6, 2.2e6, 2.8e6, 3.4e6] 
# # Fluegas Composition {columns order: H2O, CO2, N2, O2}
# FlueComp = [[0.1156, 0.020, 0.7016, 0.1628],    
#             [0.1146, 0.028, 0.6959, 0.1615],
#             [0.1137, 0.036, 0.6901, 0.1602],
#             [0.1126, 0.045, 0.6837, 0.1587]]  
# AmineConc = [0.2, 0.33, 0.46, 0.6]      
# LeanFlow = [8e+5, 1.475e+6, 2.15e+6, 2.825e+6, 3.5e+6]

FlueFlow = [1.4e6, 2.5e6, 3.2e6] 
# Fluegas Composition {columns order: H2O, CO2, N2, O2}
FlueComp = [[0.1156, 0.020, 0.7016, 0.1628],    
            [0.1137, 0.036, 0.6901, 0.1602]]  
AmineConc = [0.25, 0.4]      
LeanFlow = [1.6e6, 3.1e+6]  
LeanTemp = [33, 48, 60]
RebDuty = [245, 390, 440, 510]


iter = 0

for duty_reb in RebDuty:
    for flow_lean in LeanFlow:             # Manipulated from "MAKEUP" block
        for conc_amine in AmineConc:       # Manipulated from "MAKEUP" block
            for flow_flue in FlueFlow:
                for comp_flue in FlueComp: # Normalized composition 
                    for temp_lean in LeanTemp:
                        ## Setting the values in the simulation
                        sim.BLK_MAKEUP_Set_MassFlow(blk_makeup, flow_lean)
                        sim.BLK_MAKEUP_Set_MassFrac(blk_makeup, conc_amine, 0)
                        sim.STRM_Set_TotalFlowRate(strm_fluegas, flow_flue)
                        # sim.BLK_HEATER_Set_FlashTypeOption(blk_cooler,"TP")
                        sim.BLK_COOLER_TP_Set_Temperature(blk_cooler, temp_lean)
                        sim.BLK_RADFRAC_Set_ReboilerDuty(blk_stripper, duty_reb)
                        # FLUEGAS composition
                        sim.STRM_Set_ComponentFlowRate(strm_fluegas,comp_flue[0],"H2O")
                        sim.STRM_Set_ComponentFlowRate(strm_fluegas,comp_flue[1],"CO2")
                        sim.STRM_Set_ComponentFlowRate(strm_fluegas,comp_flue[3],"N2")   
                        sim.STRM_Set_ComponentFlowRate(strm_fluegas,comp_flue[2],"O2")
                    
                        ## Print the current iteration inputs
                        iter += 1
                        print(f"\n \niteration:{iter} "
                              f"\n FLUEGAS massflow: {flow_flue} kg/h "
                              f"\n FLUEGAS CO2 mole frac: {comp_flue[1]} "
                              f"\n LEANSOL massflow: {flow_lean} kg/h "
                              f"\n Amine concentration (wt%): {conc_amine}"
                              f"\n LEANSOL temperature: {temp_lean} C "
                              f"\n Reboiler Duty: {duty_reb} MW ")
                        
                        ## Running the simulation
                        #Suppress dialog (This is set to be True due to initialization)
                        sim.DialogSuppression(TrueOrFalse= True)
                        time_out = 70  # set the timeout to stop the simulation and move to the next iteration -> unit{seconds}
                        convergence = sim.RunWithTimeout(time_out)     
                        if convergence == True:
                            print("The simulation ran successfully.")
                        else:
                            print(f"The simulation didn't converged in {time_out}s!")
                            notConverged_itr.append(iter) # To know which iterations didn't converge
                            continue
                        
                        ## Getting the results from the simulation
                        # CO2 moleflow input (in the FLUEGAS stream)
                        moleflowCO2_fluegas_List.append(sim.STRM_Get_MoleFlowPerCompound(strm_fluegas,"CO2"))
                        # Input Amine moleflow (in the LEANSOLV stream)
                        moleflowTEA_leansolv_List.append(sim.STRM_Get_MoleFlowPerCompound(strm_leansolv,"TEA"))
                        # CO2-Amine Loading
                        loading.append(moleflowCO2_fluegas_List[-1]/ moleflowTEA_leansolv_List[-1])
                        # Fluegas Massflow
                        massflow_fluegas_List.append(flow_flue)
                        # CO2 mole fraction 
                        molefracCO2_fluegas_List.append(comp_flue[1])
                        # Amine concentration in the LEANSOLV
                        massfracAmine_leansolv_List.append(sim.STRM_Get_MassFracPerCompound(strm_leansolv,amine_comp))
                        # Lean solvent mass flow (kg/h)
                        massflow_leansolv_List.append(sim.STRM_Get_MassFlowPerCompound(strm_leansolv,amine_comp) / massfracAmine_leansolv_List[-1])
                        # Lean solvent Temperature (C)
                        temp_leansolv_List.append(temp_lean)
                        # Reboiler Duty
                        duty_reb_List.append(duty_reb)
                        # Targets
                        recovery.append(sim.CALC_Get_OutputVal("CO2REC"))
                        # Check the negativity in recovery
                        if recovery[-1] < 0:
                            print("the recovery is negative!")
                            continue
                        sec.append(sim.CALC_Get_OutputVal("ENERGY"))
                        purity.append(sim.STRM_Get_MoleFracPerCompound(strm_co2,"CO2") * 100)
                        
                        
                        # Print the targets
                        print("The CO2 Recovery%: ", recovery[-1])
                        print("The Specific Energy Consumption (kJ/kg)): ", sec[-1])
                        print("The CO2 purity%: ", purity[-1]) 
                        
                        ## Adding the data to the dictionary 
                        row = {
                            'Fluegas CO2 mol%':           100 * molefracCO2_fluegas_List[-1],
                            'Fluegas Flow (kg/h)':        massflow_fluegas_List[-1],
                            'Lean Solvent Flow (kg/h)':   massflow_leansolv_List[-1],
                            'Lean Solvent Loading':       loading[-1], # constant for now
                            'Amine type':                 amine_comp,
                            'Amine Conc (wt%)':           100 * massfracAmine_leansolv_List[-1],
                            'Lean Solvent T (C)':         temp_leansolv_List[-1],
                            'Reboiler Duty (MW)':         duty_reb_List[-1],
                            'Recovery%':                  recovery[-1],
                            'SEC (kJ/kg)':                sec[-1],
                            'CO2 Purity (mol%)':          purity[-1],
                        }
                        rows.append(row)       
                    
            
sim.CloseAspen()

## Saving to an Excel file
df = pd.DataFrame(rows)[COLUMNS]
df.to_excel(out_data, index=False)
print(f"\nSaved dataset to: {out_data}")
print(f"\n \n unconverged iterations:\n {notConverged_itr}")

The current Directory is :  
C:\Users\Pgshco\Desktop\Python ML\Carbon Capture
The new Directory where you should also have your Aspen file is : 
C:\Users\Pgshco\Desktop\Python ML\Carbon Capture
The Aspen is active now. If you dont want to see aspen open again take VISIBITLY as False 


 
iteration:1 
 FLUEGAS massflow: 1400000.0 kg/h 
 FLUEGAS CO2 mole frac: 0.02 
 LEANSOL massflow: 1600000.0 kg/h 
 Amine concentration (wt%): 0.25
 LEANSOL temperature: 33 C 
 Reboiler Duty: 245 MW 
Runtime = 20.04743981361389
The simulation ran successfully.
The CO2 Recovery%:  15.318168
The Specific Energy Consumption (kJ/kg)):  243774.897
The CO2 purity%:  2.2145637

 
iteration:2 
 FLUEGAS massflow: 1400000.0 kg/h 
 FLUEGAS CO2 mole frac: 0.02 
 LEANSOL massflow: 1600000.0 kg/h 
 Amine concentration (wt%): 0.25
 LEANSOL temperature: 48 C 
 Reboiler Duty: 245 MW 
Runtime = 5.030259609222412
The simulation ran successfully.
The CO2 Recovery%:  15.2080749
The Specific Energy Consumption (kJ/kg)):  2304

# MDEA

In [1]:
from CodeLibrary import Simulation
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Defining the name of streams and blocks in the simulation
strm_leansolv = "LEANSOLV" 
strm_cleangas = "CLEANGAS"
strm_fluegas = "FLUEGAS"
strm_co2 = "CO2"
amine_comp = "MDEA"     # Type of amine used in the simulation
blk_stripper = "STRIPPER"
blk_makeup = "MAKEUP"
blk_cooler = "COOLER"
calc_co2rec = "CO2REC" # Calculator block consist of CO2 Recovery%
calc_sec = "ENERGY"    # Calculator block consist of SEC (kJ/kg)

## Defining empty lists
moleflowCO2_fluegas_List = []   # {kmol/h}
moleflowMDEA_leansolv_List = []  # {kmol/h} 
notConverged_itr = []
# The following features are considered
molefracCO2_fluegas_List = []
massflow_fluegas_List = []      # {kg/h}
massflow_leansolv_List = []     # {kg/h}
massfracAmine_leansolv_List = []
loading = []
temp_leansolv_List = []
duty_reb_List = []
# Targets
recovery = []  # CO2 Recovery %
sec = []       # Specific Energy Comsumption (kJ/kg co2 captured)
purity = []    # CO2 Mole Fraction in the Stream Exiting the Stripper's Condenser

## Aspen and directory paths
AspenPath = r"D:/daneshgah/undergrad/Term9/Carbon Capture/Python/MDEA" # Path of the Aspen file in your system 
WorkDirect = r"C:/Users/Pgshco/Desktop/Python ML/Carbon Capture"  # The directory you are coding in
sim = Simulation(AspenFileName= AspenPath + "/MDEA.bkp", WorkingDirectoryPath= WorkDirect,VISIBILITY=True) 
# Choose the "Visibility" as "False" if you don't want the Aspen app be visible during the process

# Output dataset path
out_data = AspenPath + "/MDEAdatanew.xlsx"
# Desired column order in the Excel or CSV
COLUMNS = [
    'Fluegas CO2 mol%',
    'Fluegas Flow (kg/h)',
    'Lean Solvent Flow (kg/h)', 
    'Lean Solvent Loading',
    'Amine type',
    'Amine Conc (wt%)',
    'Lean Solvent T (C)',
    'Reboiler Duty (MW)',
    # Targets
    'Recovery%',
    'SEC (kJ/kg)',
    'CO2 Purity (mol%)',
]
rows = []

## list of features ranges in the simulation
# FlueFlow = [0.5e6, 0.9e6, 1.4e6, 2.3e6, 3.2e6] 
# # Fluegas Composition {columns order: H2O, CO2, N2, O2}
# FlueComp = [[0.1156, 0.020, 0.7016, 0.1628],    
#             [0.1146, 0.028, 0.6959, 0.1615],
#             [0.1137, 0.036, 0.6901, 0.1602],
#             [0.1126, 0.045, 0.6837, 0.1587]]  
# AmineConc = [0.2, 0.35, 0.5]      
# LeanFlow = [1.2e6, 2.1e6, 3e+6]      

FlueFlow = [1.2e6, 1.8e6, 2.9e6] 
FlueComp = [[0.1146, 0.028, 0.6959, 0.1615],
            [0.1126, 0.045, 0.6837, 0.1587]]  
AmineConc = [0.27, 0.4]      
LeanFlow = [1.6e6, 2.7e+6] 
LeanTemp = [25, 47, 60]
RebDuty = [260, 380, 420, 490]

iter = 0
for duty_reb in RebDuty:
    for flow_lean in LeanFlow:             # Manipulated from "MAKEUP" block
        for conc_amine in AmineConc:       # Manipulated from "MAKEUP" block
            for flow_flue in FlueFlow:
                for comp_flue in FlueComp: # Normalized composition 
                    for temp_lean in LeanTemp:
                        ## Setting the values in the simulation
                        sim.BLK_MAKEUP_Set_MassFlow(blk_makeup, flow_lean)
                        sim.BLK_MAKEUP_Set_MassFrac(blk_makeup, conc_amine, 0)
                        sim.STRM_Set_TotalFlowRate(strm_fluegas, flow_flue)
                        sim.BLK_COOLER_TP_Set_Temperature(blk_cooler, temp_lean)
                        sim.BLK_RADFRAC_Set_ReboilerDuty(blk_stripper, duty_reb)
                        # FLUEGAS composition
                        sim.STRM_Set_ComponentFlowRate(strm_fluegas,comp_flue[0],"H2O")
                        sim.STRM_Set_ComponentFlowRate(strm_fluegas,comp_flue[1],"CO2")
                        sim.STRM_Set_ComponentFlowRate(strm_fluegas,comp_flue[2],"N2")   
                        sim.STRM_Set_ComponentFlowRate(strm_fluegas,comp_flue[3],"O2")
                    
                        ## Print the current iteration inputs
                        iter += 1
                        print(f"\n \niteration:{iter} "
                              f"\n FLUEGAS massflow: {flow_flue} kg/h "
                              f"\n FLUEGAS CO2 mole frac: {comp_flue[1]} "
                              f"\n LEANSOL massflow: {flow_lean} kg/h "
                              f"\n Amine concentration (wt%): {conc_amine}"
                              f"\n LEANSOL temperature: {temp_lean} C "
                              f"\n Reboiler Duty: {duty_reb} MW ")
                        
                        ## Running the simulation
                        #Suppress dialog (This is set to be True due to initialization)
                        sim.DialogSuppression(TrueOrFalse= True)
                        time_out = 70  # set the timeout to stop the simulation and move to the next iteration -> unit{seconds}
                        convergence = sim.RunWithTimeout(time_out)     
                        if convergence == True:
                            print("The simulation ran successfully.")
                        else:
                            print(f"The simulation didn't converged in {time_out}s!")
                            notConverged_itr.append(iter) # To know which iterations didn't converge
                            continue
                        
                        ## Getting the results from the simulation
                        # CO2 moleflow input (in the FLUEGAS stream)
                        moleflowCO2_fluegas_List.append(sim.STRM_Get_MoleFlowPerCompound(strm_fluegas,"CO2"))
                        # Input Amine moleflow (in the LEANSOLV stream)
                        moleflowMDEA_leansolv_List.append(sim.STRM_Get_MoleFlowPerCompound(strm_leansolv,"MDEA"))
                        # CO2-Amine Loading
                        loading.append(moleflowCO2_fluegas_List[-1]/ moleflowMDEA_leansolv_List[-1])
                        # Fluegas Massflow
                        massflow_fluegas_List.append(flow_flue)
                        # CO2 mole fraction 
                        molefracCO2_fluegas_List.append(comp_flue[1])
                        # Amine concentration in the LEANSOLV
                        massfracAmine_leansolv_List.append(sim.STRM_Get_MassFracPerCompound(strm_leansolv,amine_comp))
                        # Lean solvent mass flow (kg/h)
                        massflow_leansolv_List.append(sim.STRM_Get_MassFlowPerCompound(strm_leansolv,amine_comp) / massfracAmine_leansolv_List[-1])
                        # Lean solvent Temperature (C)
                        temp_leansolv_List.append(temp_lean)
                        # Reboiler Duty
                        duty_reb_List.append(duty_reb)
                        # Targets
                        recovery.append(sim.CALC_Get_OutputVal("CO2REC"))
                        # Check the negativity in recovery
                        if recovery[-1] < 0:
                            print("the recovery is negative!")
                            continue
                        sec.append(sim.CALC_Get_OutputVal("ENERGY"))
                        purity.append(sim.STRM_Get_MoleFracPerCompound(strm_co2,"CO2") * 100)
                        
                        
                        # Print the targets
                        print("The CO2 Recovery%: ", recovery[-1])
                        print("The Specific Energy Consumption (kJ/kg)): ", sec[-1])
                        print("The CO2 purity%: ", purity[-1]) 
                        
                        ## Adding the data to the dictionary 
                        row = {
                            'Fluegas CO2 mol%':           100 * molefracCO2_fluegas_List[-1],
                            'Fluegas Flow (kg/h)':        massflow_fluegas_List[-1],
                            'Lean Solvent Flow (kg/h)':   massflow_leansolv_List[-1],
                            'Lean Solvent Loading':       loading[-1], # constant for now
                            'Amine type':                 amine_comp,
                            'Amine Conc (wt%)':           100 * massfracAmine_leansolv_List[-1],
                            'Lean Solvent T (C)':         temp_leansolv_List[-1],
                            'Reboiler Duty (MW)':         duty_reb_List[-1],
                            'Recovery%':                  recovery[-1],
                            'SEC (kJ/kg)':                sec[-1],
                            'CO2 Purity (mol%)':          purity[-1],
                        }
                        rows.append(row)       
                    
            
sim.CloseAspen()

## Saving to an Excel file
df = pd.DataFrame(rows)[COLUMNS]
df.to_excel(out_data, index=False)
print(f"\nSaved dataset to: {out_data}")
print(f"\n \n unconverged iterations:\n {notConverged_itr}")

The current Directory is :  
C:\Users\Pgshco\Desktop\Python ML\Carbon Capture
The new Directory where you should also have your Aspen file is : 
C:\Users\Pgshco\Desktop\Python ML\Carbon Capture
The Aspen is active now. If you dont want to see aspen open again take VISIBITLY as False 


 
iteration:1 
 FLUEGAS massflow: 1200000.0 kg/h 
 FLUEGAS CO2 mole frac: 0.028 
 LEANSOL massflow: 1600000.0 kg/h 
 Amine concentration (wt%): 0.27
 LEANSOL temperature: 25 C 
 Reboiler Duty: 260 MW 
Runtime = 45.018332958221436
The simulation ran successfully.
The CO2 Recovery%:  30.0519779
The Specific Energy Consumption (kJ/kg)):  107419.906
The CO2 purity%:  5.87498639

 
iteration:2 
 FLUEGAS massflow: 1200000.0 kg/h 
 FLUEGAS CO2 mole frac: 0.028 
 LEANSOL massflow: 1600000.0 kg/h 
 Amine concentration (wt%): 0.27
 LEANSOL temperature: 47 C 
 Reboiler Duty: 260 MW 
Runtime = 15.231637477874756
The simulation ran successfully.
The CO2 Recovery%:  30.0977719
The Specific Energy Consumption (kJ/kg)):

KeyError: "['Lean Solvent T (C)', 'Reboiler Duty (MW)'] not in index"